In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
os.environ["SPARK_HOME"]='/opt/cloudera/parcels/SPARK2/lib/spark2/'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.6-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())
print()

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.cloudera2
      /_/

Using Python version 3.4.3 (default, Nov 17 2016 01:08:31)
SparkSession available as 'spark'.


In [91]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark.sql.functions as f

@f.udf(ArrayType(StringType()))
def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        
        ids = [i for i in range(1, len(words)) if words[i-1].lower() == "narodnaya"]
        return ["narodnaya_" + words[i].lower() for i in ids]
    except ValueError as e:
        return []

In [92]:
# wiki = sc.textFile("hdfs:///data/wiki/en_articles_part1/articles-part", 16).map(parse_article)
log_schema = StructType(fields=[
    StructField("content", StringType())
])
wiki = spark.read.csv("hdfs:///data/wiki/en_articles", schema=log_schema).cache()

In [93]:
# wiki.take(1)
# parse_article('12\tAnarchism         Anarchism is narodnaya defined narodnaya a political narodnaya which holds the state to be undesirable')

In [94]:
wiki = wiki.withColumn("words_list", parse_article("content"))

In [96]:
ans = wiki.select(f.explode("words_list").alias("word")).groupBy("word").count().orderBy("word").collect()
# wiki.filter(f.col("content").like("%Narodnay%")).take(10)

for (word, c) in ans:
    print(word + "\t" + str(c))


narodnaya_also	1
narodnaya_eurasia	1
narodnaya_gazeta	2
narodnaya_mount	1
narodnaya_narodnaya	1
narodnaya_rasprava	1
narodnaya_romanization	1
narodnaya_sovetskaya	1
narodnaya_volya	10
narodnaya_volya.after	1


In [97]:
sc.stop()